In [12]:
from kafka import KafkaConsumer
from time import sleep
from json import dumps,loads
import json
from s3fs import S3FileSystem

In [14]:
import time
import boto3
import json

def upload_to_s3(bucket_name, file_name, data):
    """
    Upload data to S3 using boto3.
    :param bucket_name: Name of the S3 bucket.
    :param file_name: Name of the file to upload.
    :param data: Data to write (must be JSON serializable).
    """
    s3 = boto3.client("s3")
    s3.put_object(
        Bucket=bucket_name,
        Key=file_name,
        Body=json.dumps(data),
        ContentType="application/json"
    )
    print(f"File uploaded to s3://{bucket_name}/{file_name}")
    
consumer = KafkaConsumer(
    'demo_testing2',
    bootstrap_servers=['{Public IP}:9092'],
    value_deserializer=lambda x: json.loads(x.decode('utf-8')),
)

# S3 setup
s3 = S3FileSystem()
def consume_and_store(consumer, s3):
    """
    Consume data from Kafka and store it in S3.
    """
    for message in consumer:
        if message:
            data = message.value
            print(f"Consumed from Kafka: {data['topic_name']}")

            # Save to S3
            file_path = f"s3://bucket name/eksisozluk_{int(time.time())}.json"
            # with s3.open(file_path, 'w') as file:
            #     json.dump(data, file)
            upload_to_s3('{bucket name}', f'eksisozluk_{int(time.time())}.json', data)
            print(f"Saved to S3: {file_path}")
            time.sleep(0.3)
        else:
            print("NO DATA")

        # Stop consuming after one message (remove break for continuous consumption)

In [ ]:
consume_and_store(consumer=consumer,s3=s3)